In [ ]:
os.chdir('InsightFace_Pytorch/')

In [ ]:
mtcnn = MTCNN()

In [ ]:
filenames = []
for root, dirs, files in os.walk("../dataset/CelebAMask-HQ/CelebA-HQ-img/"):
    for name in files:
        if name.endswith('jpg') or name.endswith('png'):
            p = os.path.join(root, name)
            filenames.append(p)

In [ ]:
p = filenames[22]

In [ ]:
img = cv2.imread(p)[:, :, ::-1]

In [ ]:
Image.fromarray(img)

In [ ]:
faces = mtcnn.align_multi(Image.fromarray(img), min_face_size=64, crop_size=(256, 256))

In [ ]:
faces

In [ ]:
faces[0].size

In [ ]:
faces[0]

In [ ]:

model = Backbone(50, 0.6, 'ir_se')#.to(device)
model.eval()
model.load_state_dict(torch.load('model_ir_se50.pth', map_location='cpu'))


In [ ]:
test_transform = trans.Compose([
    trans.ToTensor(),
    trans.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [ ]:
for face in faces:
    scaled_img = face.resize((112, 112), Image.ANTIALIAS)
    with torch.no_grad():
        embed = model(test_transform(scaled_img).unsqueeze(0)).squeeze().cpu().numpy()

In [ ]:
embed.shape

In [ ]:
from torch.utils.data import DataLoaderimport sys

sys.path.append('InsightFace_Pytorch/')

import torch
from mtcnn import MTCNN
import cv2
import numpy as np

import PIL.Image as Image
from model import Backbone, Arcface, MobileFaceNet, Am_softmax, l2_norm
from torchvision import transforms as trans
import os



from torch.utils.data import TensorDataset

import random

import torchvision.transforms as transforms

In [ ]:

class FaceDatasetNoPrep(TensorDataset):
    def __init__(self, image_folder=image_folder, same_prob=same_prob):
        self.same_prob = same_prob
        
        self.filenames = []
        for root, dirs, files in os.walk(image_folder):
            for name in files:
                if name.endswith('jpg') or name.endswith('png'):
                    p = os.path.join(root, name)
                    self.filenames.append(p)
        
#         self.mtcnn = MTCNN()
        
        
        self.transforms = transforms.Compose([
            transforms.ColorJitter(0.2, 0.2, 0.2, 0.01),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, item):
        X_s = cv2.imread(self.filenames[item])[:, :, ::-1]
        if random.random() > self.same_prob:
            random_target_img_idx = random.randint(0, len(self.filenames)-1)
            image_path = self.filenames[random_target_img_idx]
            Xt = cv2.imread(image_path)[:, :, ::-1]
            
            same_person = 0
            
            return self.transforms(X_s), self.transforms(X_t), same_person
        
        else:
            Xt = X_s.copy()
            same_person = 1
            
            return self.transforms(X_s), self.transforms(X_t), same_person

In [ ]:
image_folder = "dataset/CelebAMask-HQ/CelebA-HQ-img/"
same_prob = 0.2

class FaceDataset(TensorDataset):
    def __init__(self, image_folder=image_folder, same_prob=same_prob):
        self.same_prob = same_prob
        
        self.filenames = []
        for root, dirs, files in os.walk(image_folder):
            for name in files:
                if name.endswith('jpg') or name.endswith('png'):
                    p = os.path.join(root, name)
                    self.filenames.append(p)
        
        self.mtcnn = MTCNN()
        
        
        self.transforms = transforms.Compose([
            transforms.ColorJitter(0.2, 0.2, 0.2, 0.01),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])
        
    def __len__(self):
        return len(self.filenames)
    def __getitem__(self, item):
        img = cv2.imread(self.filenames[item])[:, :, ::-1]
        print('get')
        if random.random() > self.same_prob:

            random_target_img_idx = random.randint(0, len(self.filenames)-1)
            image_path = self.filenames[random_target_img_idx]
            Xt = cv2.imread(image_path)[:, :, ::-1]

            same_person = 0
            X_s = self.align_and_crop(img)
            X_t = self.align_and_crop(Xt)
            
            if not X_s:
                item += 1
                return self.__getitem__(item)
            if not X_t:
                return self.__getitem__(item)
            return self.transforms(X_s), self.transforms(X_t), same_person
        else:
            Xt = img.copy()
            same_person = 1

            X_s = self.align_and_crop(img)
            X_t = self.align_and_crop(Xt)
            if not X_s:
                item += 1
                return self.__getitem__(item)
            if not X_t:
                return self.__getitem__(item)
            return self.transforms(X_s), self.transforms(X_t), same_person
        
        
    def align_and_crop(self, img):
        faces = self.mtcnn.align_multi(Image.fromarray(img), min_face_size=64, crop_size=(256, 256))
        if not faces:
            return False

        return faces[0]

In [ ]:
source_folder = "dataset/CelebAMask-HQ/CelebA-HQ-img/"
target_folder = "dataset/celeb_cropped"

In [ ]:
dl = FaceDataset()

In [ ]:
for root, dirs, files in os.walk(image_folder):
    for name in files:
        if name.endswith('jpg') or name.endswith('png'):
            p = os.path.join(root, name)
            img = cv2.imread(p)[:, :, ::-1]
            result = dl.align_and_crop(img)
            if not result:
                continue
            result.save(os.path.join(target_folder, name))

In [ ]:
Image.wr

In [ ]:
name

In [ ]:
p

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 32

In [ ]:
DL = DataLoader(dl, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

In [ ]:
for data in DL:
    break

In [ ]:
X_s, X_t, same = data

In [ ]:
X_s.shape, X_t.shape, same

In [ ]:
import os

In [1]:
from DataLoader import FaceDatasetNoPrep

In [33]:
dataset = FaceDatasetNoPrep()

In [34]:
len(dataset)

29539

In [4]:
from torch.utils.data import DataLoader

In [9]:
batch_size = 32

In [10]:
dl = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

In [11]:
for data in dl:
    break
    
print(data[0].shape, data[1].shape, data[2])

torch.Size([32, 3, 256, 256]) torch.Size([32, 3, 256, 256]) tensor([1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 0, 1])


In [12]:
import shutil

In [13]:
import os

In [15]:
images = os.listdir('dataset/celeb_cropped/')

In [16]:
filepathes = [os.path.join('dataset/celeb_cropped', image) for image in images]

In [17]:
import numpy as np

In [18]:
len(filepathes)

29613

In [29]:
valid = np.random.choice(images, size = int(len(images) * 0.03) )

In [30]:
len(valid)

888

for image in valid:
    shutil.move(os.path.join('dataset/celeb_cropped', image), os.path.join('dataset/celeb_cropped_valid', image))